In [23]:
import pandas as pd
from bs4 import BeautifulSoup
import json

# Step 1: Read the CSV file
csv_file = 'insight.csv'
df_csv = pd.read_csv(csv_file)

# Initialize a dictionary to hold all table data, with major categories as keys
all_tables_data = {}

# Iterate over each item in the row from columns 'B' to 'P' (indices 1 to 15)
for column_index in range(1, 16):
    # Extract HTML content from the specified cell (second row, index 2, and current column)
    html_content = df_csv.iloc[1, column_index]
    # Step 2: Parse the HTML content to extract table data
    soup = BeautifulSoup(html_content, 'html.parser')
    if column_index == 1:
        table_id = 'DataTables_Table_31'
    else:
        table_id = f'DataTables_Table_{column_index+1}'

    table = soup.find('table', {'id': table_id})

    # Proceed only if a table is found
    if table:
        # Extract headings
        headings = [th.get_text() for th in table.find('thead').find_all('th')]

        # Extract rows
        rows = table.find('tbody').find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            row_data = dict(zip(headings, cols))
            
            # Check if Mosaic is a major category or subcategory
            mosaic = row_data['Mosaic']
            if mosaic.isalpha():  # Major category
                if mosaic not in all_tables_data:
                    all_tables_data[mosaic] = []
                # Remove Mosaic key for major category
                del row_data['Mosaic']
                all_tables_data[mosaic].append(row_data)
            else:  # Subcategory
                major_category = mosaic[0]
                if major_category not in all_tables_data:
                    all_tables_data[major_category] = []
                all_tables_data[major_category].append(row_data)

# Step 3: Convert the aggregated data into a JSON format
data_json = json.dumps(all_tables_data, indent=4)

# Output JSON
print(data_json)

{
    "A": [
        {
            "Name": "City Prosperity",
            "Description": "High status city dwellers living in central locations and pursuing careers with high rewards",
            "Electors": "4.4%",
            "Key Feature 1": "Urban areas",
            "Key Feature 2": "High value flats",
            "Key Feature 3": "High income",
            "Key Feature 4": "Cosmopolitan lifestyles",
            "Key Feature 5": "High-tech homes",
            "Key Feature 6": "Uber passengers"
        },
        {
            "Mosaic": "A01",
            "Name": "World-Class Wealth",
            "Description": "Global high flyers and moneyed families living luxurious lifestyles in London's most exclusive boroughs",
            "Electors": "0.6%",
            "Key Feature 1": "Expensive properties",
            "Key Feature 2": "5+ bedrooms",
            "Key Feature 3": "High income",
            "Key Feature 4": "University graduates",
            "Key Feature 5": "High-tech hom

In [21]:
import pandas as pd
import json

# Read the CSV file
csv_file = 'insight.csv'
df_csv = pd.read_csv(csv_file)

# Initialize a list to hold dictionaries for each row's structured data
structured_data = []

# Iterate over each item in the row from columns 'B' to 'P' (indices 1 to 15)
for column_index in range(1, 16):
    # Extract text content from the specified cell (first row, index 1, and current column)
    text_content = df_csv.iloc[0, column_index]
    
    # Split the text content into lines
    lines = text_content.split('\n')
    
    # Initialize a temporary dictionary to hold the current column's data
    temp_dict = {}
    
    # Use the first line as the first key:value pair, splitting on ":"
    first_line = lines[0].split(':')
    if len(first_line) == 2:
        temp_dict[first_line[0].strip()] = first_line[1].strip()
    
    # Start iterating from the second line
    current_key = None
    for line in lines[1:]:
        if not line.startswith("UK:"):
            if line.isupper():
                current_key = line
            else:
                if current_key:
                    temp_dict[current_key] = line.strip()
                    current_key = None
                else:
                    if temp_dict:
                        last_key = list(temp_dict.keys())[-1]
                        temp_dict[last_key] += ' ' + line.strip()
    
    # Append the temporary dictionary to the structured_data list
    structured_data.append(temp_dict)

# Manually remove trailing whitespaces from each value
for item in structured_data:
    for key in item:
        item[key] = item[key].rstrip()

# Convert the structured data into JSON format
data_json = json.dumps(structured_data, indent=4)

# Output JSON
print(data_json)

[
    {
        "A": "City Prosperity (4.4%)",
        "TENURE": "59.8% Owned",
        "MEDIAN PERSONAL INCOME": "\u00a358500",
        "FAMILY TYPE": "20.4% Older family no children <18",
        "AGE": "21.2% 26-35",
        "CHANNEL PREFERENCE": "Mobile",
        "EDUCATION LEVEL": "56.9% University degree or higher",
        "AVERAGE LABOUR SCORE": "50",
        "AVERAGE TURNOUT SCORE": "66",
        "TIME TO KNOCK": "Weekend"
    },
    {
        "B": "Prestige Positions (9.2%)",
        "TENURE": "95.3% Owned",
        "MEDIAN PERSONAL INCOME": "\u00a337500",
        "FAMILY TYPE": "34.9% Older family no children <18",
        "AGE": "29.6% 66+",
        "CHANNEL PREFERENCE": "Internet",
        "EDUCATION LEVEL": "76.2% University degree or higher",
        "AVERAGE LABOUR SCORE": "36",
        "AVERAGE TURNOUT SCORE": "83",
        "TIME TO KNOCK": "Weekend"
    },
    {
        "C": "Country Living (6.3%)",
        "TENURE": "90.5% Owned",
        "MEDIAN PERSONAL INCOME": "\

In [ ]:
{
  "TENURE": "Predicted ownership type of property - owned, private rented or socially rented (council or housing association)",
  "MEDIAN_PERSONAL_INCOME": "Predicted median income of this group",
  "FAMILY_TYPE": "Modal family type - rough age and whether any children",
  "AGE": "Modal estimated age of members of this group",
  "CHANNEL_PREFERNCE": "Does this Group overindex compared to the national average for any particular contact method?",
  "EDUCATION_LEVEL": "Predicted level of education achieved. No qualifications up to University degree",
  "AVERAGE_LABOUR_SCORE": "Modelled score predicting, for every elector, their likelihood to support Labour",
  "AVERAGE_TURNOUT_SCORE": "Modelled score predicting, for every elector, their likelihood to turn out to vote",
  "TIME_TO_KNOCK": "While the evening is the best time overall, does this group overindex compared to the average for a particular time of day?"
}
